In [1]:
from util import *
from preprocess import *
import os
from tqdm.notebook import tqdm
import torch.nn.functional as F

WORD_IMG_SHAPE = (64, 64 * 4)
LATENT_SIZE = np.prod(WORD_IMG_SHAPE)
BATCH_SIZE = 32

In [2]:
generator = nn.Sequential(
    # in: latent_size x 1 x 1

    nn.ConvTranspose2d(LATENT_SIZE, 1024, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(1024),
    nn.ReLU(True),
    # out: 512 x 4 x 4

    nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 256 x 8 x 8

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 128 x 16 x 16

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 64 x 32 x 32
    
    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # out: 64 x 32 x 32

    nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh(),
    # out: 1 x 64 x 64
    
    Reshape('0', 1, '2*3')
)

# generator = nn.Sequential(
#     # in: latent_size x 1 x 1

#     nn.ConvTranspose2d(LATENT_SIZE, 2048, kernel_size=4, stride=1, padding=0, bias=False),
#     nn.BatchNorm2d(2048),
#     nn.ELU(True),
#     # out: 2048 x 4 x 4

#     nn.ConvTranspose2d(2048, 1024, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.BatchNorm2d(1024),
#     nn.ELU(True),
#     # out: 1024 x 8 x 8

#     nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.BatchNorm2d(512),
#     nn.ELU(True),
#     # out: 512 x 16 x 16
    
#     nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.MaxPool2d(kernel_size=4, stride=2, padding=1),
#     nn.BatchNorm2d(512),
#     nn.ELU(True),
#     # out: 512 x 16 x 16

#     nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.BatchNorm2d(256),
#     nn.ELU(True),
#     # out: 256 x 32 x 32
    
#     nn.ConvTranspose2d(256, 256, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.MaxPool2d(kernel_size=4, stride=2, padding=1),
#     nn.BatchNorm2d(256),
#     nn.ELU(True),
#     # out: 256 x 32 x 32
    
#     nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.BatchNorm2d(128),
#     nn.ELU(True),
#     # out: 128 x 64 x 64
    
#     nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.BatchNorm2d(64),
#     nn.MaxPool2d(kernel_size=4, stride=2, padding=1),
#     nn.ELU(True),
#     # out: 64 x 64 x 64

#     nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1, bias=False),
#     nn.Tanh(),
#     # out: 1 x 64 x 64
    
#     Reshape('0', 1, '2*3')
# )

In [3]:
# char_model = load_char_restoration_model()
# # print(char_model.named_parameters)
# # print(generator.named_parameters)
# # generator.no_grad()
# # char_model.no_grad()
# generator = copy_weights(char_model, generator)
# generator.load_state_dict(torch.load('./trained/word/full/generator'))

In [4]:
sample_dir = 'generated'
os.makedirs(sample_dir, exist_ok=True)

def save_samples(index, latent_tensors, show=True, num_samples=3*8):
    fake_images = generator(latent_tensors)
    fake_fname = 'generated-images-{0:0=4d}.png'.format(index)
    save_image(fake_images.reshape((num_samples, 1, *WORD_IMG_SHAPE)), os.path.join(sample_dir, fake_fname), nrow=3, pad_value=1)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=12).permute(1, 2, 0))

In [5]:


def train_generator(opt_g, base_imgs, crpt_imgs):
    batch_size = base_imgs.shape[0]
    # Clear generator gradients
    opt_g.zero_grad()
    
    # Generate fake images
    # latent = torch.randn(batch_size, latent_size, 1, 1)
    # corrupted, base, labels = next(cwg)
    # print(np.shape(crpt_imgs))
    # print(np.shape(crpt_imgs[0]))
    # print(type(crpt_imgs))
    # print(crpt_imgs.reshape((BATCH_SIZE, LATENT_SIZE, 1, 1)).shape)
    
    rest_imgs = generator(crpt_imgs.reshape((batch_size, LATENT_SIZE, 1, 1)))
    # rest_imgs = generator(base_imgs.reshape((batch_size, LATENT_SIZE, 1, 1)))
   
    # print(f"rest img shape: {rest_imgs.shape}")
    # Try to fool the discriminator
    # fool_preds = discriminator(fake_images)
    # fool_targets = torch.ones(batch_size, 1)
    # fool_loss = F.mse_loss(fool_preds, fool_targets)
    
    sim_loss = F.mse_loss(rest_imgs / 2, (base_imgs / 2).reshape((batch_size, 1, LATENT_SIZE)))
    # print(f"loss shape: {sim_loss.shape}")
    # black_loss = F.l1_loss(torch.zeros(rest_imgs.shape), (-rest_imgs + 1) / 2)
    # print(base_imgs.shape)
    # print(rest_imgs.shape)
    ink_loss = F.mse_loss(torch.zeros(rest_imgs.shape), 
                          (img_subtract(base_imgs.reshape(rest_imgs.shape), rest_imgs) + 1) / 2)
    
    # Update generator weights
    # loss = 2.5 * fool_loss + 150 * sim_loss
    loss = 50 * sim_loss + 150 * ink_loss
    loss.backward()
    opt_g.step()
    
    print(f"generator loss: {(torch.mean(loss).item())}")
    
    return loss.item()

In [6]:
num_examples = 3*8

data_loader = DataLoader(load_dataset('char', equal_shapes=False), shuffle=True, num_workers=0, pin_memory=True)
cwg = CorruptWordGen(data_loader, batch_size=128, img_shape=WORD_IMG_SHAPE)

fixed_base, fixed_corrupted, fixed_labels = next(cwg)
fixed_corrupted = fixed_corrupted.reshape((128, LATENT_SIZE, 1, 1))

data_loader = DataLoader(load_dataset('char', equal_shapes=False), shuffle=True, num_workers=0, pin_memory=True)
cwg = CorruptWordGen(data_loader, batch_size=num_examples, img_shape=WORD_IMG_SHAPE)

target_distances = []
crpt_distances = []

def fit(epochs, lr, start_idx=1):
  torch.cuda.empty_cache()
    
  # Losses & scores
  losses_g = []
  losses_d = []
  real_scores = []
  fake_scores = []
  gen_scores = []
  
  # Create optimizers
  # opt_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
  opt_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
  
  for epoch in range(epochs):
    target_distances.append([])
    crpt_distances.append([])
    
    data_loader = DataLoader(load_dataset('char', equal_shapes=False), shuffle=True, num_workers=0, pin_memory=True)
    cwg = CorruptWordGen(data_loader, batch_size=BATCH_SIZE, img_shape=WORD_IMG_SHAPE)
    
    for base_imgs, crpt_imgs, labels in tqdm(cwg):
      
      # # Train discriminator
      # loss_d, real_score, fake_score = train_discriminator(real_images, opt_d)
      # # Train generator
      loss_g = train_generator(opt_g, base_imgs, crpt_imgs)
      
      fixed_predicted = generator(fixed_corrupted)
      target_dist = img_distance(fixed_predicted.detach().numpy(), fixed_base.detach().numpy())
      crpt_dist = img_distance(fixed_predicted.detach().numpy(), fixed_corrupted.detach().numpy())
      
      print(f"td: {target_dist} | cd: {crpt_dist}")
      target_distances[-1].append(float(target_dist))
      crpt_distances[-1].append(float(crpt_dist))
      
      with open('./trained/word/scores.json', "w+") as file:
        json.dump({"target": target_distances, "corrupt": crpt_distances}, file)
        
    # Record losses & scores
    losses_g.append(loss_g)
    # losses_d.append(loss_d)
    # real_scores.append(real_score)
    # fake_scores.append(fake_score)
    
    # Log losses & scores (last batch)
    print("Epoch [{}/{}], loss_g: {:.4f}".format(
      epoch+1, epochs, loss_g))

    # Save generated images
    save_samples(epoch+start_idx, fixed_corrupted, show=False, num_samples = 128)
    
    torch.save(generator.state_dict(), "./trained/word/generator")
    torch.save(opt_g.state_dict(), './trained/word/opt')
    try:
      torch.save(generator.state_dict(), f"./trained/word/{epoch}/generator")
      torch.save(opt_g.state_dict(), f'./trained/word/{epoch}/opt')
    except:
      pass
  
  return losses_g, losses_d, real_scores, fake_scores

In [7]:

# print(torch.cat(fixed_base).shape)
# print(np.shape(fixed_base[0]))
# print(np.shape(fixed_corrupted[0]))
save_image(fixed_base.reshape((128, 1, *WORD_IMG_SHAPE)), os.path.join('generated', "base.png"), nrow=3, pad_value=1)
save_image(fixed_corrupted.reshape((128, 1, *WORD_IMG_SHAPE)), os.path.join('generated', "corrupted.png"), nrow=3, pad_value=1)
print(fixed_labels)

[[21, 20, 20, 2, 10, 16, 7, 12], [9, 11, 4, 4, 18], [4, 19], [1, 6, 12, 10, 22, 16], [5, 3, 9, 21], [4, 5, 10, 2, 20, 4, 4, 20, 10], [23, 16, 1, 6, 0, 16], [6, 23], [11, 3, 7, 5, 10], [23, 7], [10, 19], [10, 20, 9, 6, 2, 19, 10, 17, 1], [18, 16, 10, 1, 4, 18], [16, 23, 9, 16, 18], [0, 24, 17, 1, 23, 5, 3], [9, 11, 19, 6, 19, 6, 20, 6], [10, 11, 23, 13, 13], [21, 18, 1, 20, 0, 13, 13, 1, 0], [4, 9, 21, 18], [13, 17, 19, 2, 4, 6, 3, 24, 2], [5, 12, 16, 12, 20, 4, 0], [0, 2], [4, 2, 1, 11, 22, 18, 19], [25, 0, 10, 21, 16, 22, 6, 7, 7], [11, 6, 5], [13, 18, 2, 3, 20, 4, 23, 13, 10], [10, 13, 17, 3, 21, 18], [11, 11, 4, 9, 1, 8], [7, 23, 1, 16, 11, 18, 23], [1, 1, 18, 9, 5, 18], [18, 23, 7, 11, 23, 13, 19], [18, 9, 21, 24, 9], [9, 1, 19, 16], [19, 10, 18, 5, 11, 19, 5, 9, 16], [21, 4], [5, 6, 13, 13, 2, 0, 23], [5, 10, 1, 5], [1, 7, 5, 12], [5, 19, 23, 2], [18, 6, 13, 6, 2, 18, 21, 9, 10], [20, 11, 16, 10, 13, 5, 18], [3, 10, 19, 6, 10], [20, 10, 2, 2, 22, 9, 20, 4, 23], [18, 9, 23, 1], [18

In [8]:
save_samples(0, fixed_corrupted, show=False, num_samples = 128)
history = fit(100, 3e-3)

generator loss: 9.098737716674805
td: 0.48424240946769714 | cd: 0.5123311281204224
generator loss: 8.79001522064209
td: 0.4804728329181671 | cd: 0.508025586605072
generator loss: 8.932100296020508
td: 0.4755464792251587 | cd: 0.5014597773551941
generator loss: 8.727899551391602
td: 0.4712120592594147 | cd: 0.4950788617134094
generator loss: 9.029197692871094
td: 0.4753841757774353 | cd: 0.49936801195144653
generator loss: 8.719356536865234
td: 0.477647989988327 | cd: 0.49646204710006714
generator loss: 8.474853515625
td: 0.4756593108177185 | cd: 0.491875022649765
generator loss: 8.125402450561523
td: 0.46560725569725037 | cd: 0.4800947308540344
generator loss: 9.357073783874512
td: 0.46906331181526184 | cd: 0.4827301800251007
generator loss: 8.641459465026855
td: 0.4736863374710083 | cd: 0.4914219379425049
generator loss: 8.642958641052246
td: 0.4772891700267792 | cd: 0.4977235794067383
generator loss: 9.393062591552734
td: 0.48901352286338806 | cd: 0.5110732316970825
generator loss: 8

0it [00:00, ?it/s]

generator loss: 8.450346946716309
td: 0.4783462584018707 | cd: 0.5060997605323792
generator loss: 9.939634323120117
td: 0.4944142997264862 | cd: 0.5235000848770142
generator loss: 9.187490463256836
td: 0.5038027763366699 | cd: 0.5374012589454651
generator loss: 8.949199676513672
td: 0.5056130290031433 | cd: 0.5390403270721436
generator loss: 8.83380126953125
td: 0.49576929211616516 | cd: 0.5261449217796326
generator loss: 8.898027420043945
td: 0.4898679256439209 | cd: 0.5148878693580627
generator loss: 8.294046401977539
td: 0.4849952161312103 | cd: 0.5076574683189392
generator loss: 7.792529106140137
td: 0.4663355350494385 | cd: 0.4893123209476471
generator loss: 7.936770915985107
td: 0.456940233707428 | cd: 0.48238080739974976
generator loss: 8.378988265991211
td: 0.4595581293106079 | cd: 0.48469123244285583
generator loss: 8.649188041687012
td: 0.4663674831390381 | cd: 0.4925096333026886
generator loss: 9.11888599395752
td: 0.4796242117881775 | cd: 0.5110548138618469
generator loss: 

0it [00:00, ?it/s]

generator loss: 8.122068405151367
td: 0.4673268496990204 | cd: 0.48841068148612976
generator loss: 8.968172073364258
td: 0.4661701023578644 | cd: 0.48494502902030945
generator loss: 9.268107414245605
td: 0.4792930781841278 | cd: 0.497877299785614
generator loss: 9.430583953857422
td: 0.4894193112850189 | cd: 0.5132365226745605
generator loss: 9.040639877319336
td: 0.4913107752799988 | cd: 0.5204225182533264
generator loss: 8.900372505187988
td: 0.4897080361843109 | cd: 0.5154680609703064
generator loss: 9.356162071228027
td: 0.49830490350723267 | cd: 0.5233957171440125
generator loss: 9.953516960144043
td: 0.5068017244338989 | cd: 0.5340191721916199
generator loss: 8.980924606323242
td: 0.49936455488204956 | cd: 0.5281204581260681
generator loss: 8.88220500946045
td: 0.4861682057380676 | cd: 0.5173124670982361
generator loss: 9.838462829589844
td: 0.49169284105300903 | cd: 0.5195626616477966
generator loss: 8.761393547058105
td: 0.4898340702056885 | cd: 0.5145488381385803
generator los

...

generator loss: 9.830411911010742
td: 0.4893091917037964 | cd: 0.5163015127182007
generator loss: 9.103815078735352
td: 0.48894333839416504 | cd: 0.5134021639823914
generator loss: 8.89426040649414
td: 0.47914043068885803 | cd: 0.5018270015716553
generator loss: 9.714180946350098
td: 0.4864473342895508 | cd: 0.5075770020484924
generator loss: 9.287596702575684
td: 0.49629706144332886 | cd: 0.5188658237457275
generator loss: 8.588191986083984
td: 0.49420782923698425 | cd: 0.5164819955825806
generator loss: 8.882783889770508
td: 0.49236297607421875 | cd: 0.5131354928016663
generator loss: 9.548439979553223
td: 0.48937588930130005 | cd: 0.51172935962677
generator loss: 8.726018905639648
td: 0.4861747622489929 | cd: 0.5121686458587646
generator loss: 8.54312801361084
td: 0.4782361686229706 | cd: 0.5016577243804932
generator loss: 8.028617858886719
td: 0.468013733625412 | cd: 0.49447816610336304
generator loss: 9.055866241455078
td: 0.4693821370601654 | cd: 0.501743495464325
generator loss:

0it [00:00, ?it/s]

generator loss: 9.027277946472168
td: 0.48303458094596863 | cd: 0.5113848447799683
generator loss: 9.236903190612793
td: 0.49003809690475464 | cd: 0.5217920541763306
generator loss: 8.665534019470215
td: 0.4823676347732544 | cd: 0.5170688629150391
generator loss: 8.716695785522461
td: 0.4758214056491852 | cd: 0.5110942721366882
generator loss: 9.321504592895508
td: 0.47863271832466125 | cd: 0.509991466999054
generator loss: 8.157994270324707
td: 0.47271645069122314 | cd: 0.4957320988178253
generator loss: 7.767993927001953
td: 0.4610721170902252 | cd: 0.48541060090065
generator loss: 8.575085639953613
td: 0.4610019326210022 | cd: 0.48580291867256165
generator loss: 8.667200088500977
td: 0.4654262363910675 | cd: 0.4888211190700531
generator loss: 9.073576927185059
td: 0.4705928862094879 | cd: 0.4953836500644684
generator loss: 9.24389362335205
td: 0.4815165400505066 | cd: 0.5128661394119263
generator loss: 9.075593948364258
td: 0.497572124004364 | cd: 0.5294358134269714
generator loss: 

: 

: 